In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, '../')

import pandas as pd
import xlsxwriter
import os
import os.path as osp
import numpy as np

#my code
from load_utils import loadExcel
from sheet_number_of_travels import createSheetNumberOfTravels
from sheet_times import createSheetTimes
from sheet_number_of_cars import createSheetNumberOfCars, computeData, createFakeDataset
import yaml

In [27]:
with open('../parameters.yaml') as f:
    params = yaml.safe_load(f)
N = params['number_of_cameras']

df_save = loadExcel('input.xlsx', N=params['number_of_cameras'])

In [28]:
df = df_save.copy(deep=True)
selectedDirections = [1,3,5]

In [38]:
writer.close()
if osp.isfile('out.xlsx'):
    os.remove('out.xlsx')

with open('../parameters.yaml') as f:
    params = yaml.safe_load(f)
    
    writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
    createSheetNumberOfTravels(df.copy(True), writer, params)
    writer.close()


In [83]:
writer.close()

In [63]:
abc = createFakeDataset(N)
abc

,Capture_time,License_plate,Direction
0,2021-11-14 00:00:00,fake_1,1
1,2021-11-14 00:00:00,fake_1_1,1
2,2021-11-14 01:00:00,fake_1_1,1
3,2021-11-14 00:00:00,fake_1_2,1
4,2021-11-14 01:00:00,fake_1_2,2
...,...,...,...
661,2021-11-14 01:00:00,fake_18_16,16
662,2021-11-14 00:00:00,fake_18_17,18
663,2021-11-14 01:00:00,fake_18_17,17
664,2021-11-14 00:00:00,fake_18_18,18


In [ ]:
crosstab = pd.crosstab(df_multiple_join['Direction_from'],
                           df_multiple_join['Direction_to'])

In [27]:
def computeDataXD(df, params, startTime='00:00', endTime='23:59'):
    # filter time
    df = df.set_index('Capture_time').between_time(startTime, endTime).reset_index()

    # add fake data to force dataframe layout, aka have at least one entry for every possible combination
    fake_df = createFakeDataset(params['number_of_cameras'])
    df = df.append(fake_df, ignore_index=True)

    # separate dataset into two parts based on license plate count
    plate_counts = df["License_plate"].value_counts()
    plate_counts['unknown'] = 0  # fake unknown values, so they are selected into single dataset
    plate_count = plate_counts[plate_counts > 1]

    df_multiple = df[df['License_plate'].isin(plate_count.index)]
    return df_multiple

    # create another dataset moved by one and merge it. Aka create pairs of all following directions
    df_multiple = df_multiple.sort_values(by=['License_plate', 'Capture_time']).reset_index(drop=True)
    df_multiple['joinIndex'] = df_multiple.index
    df_multiple_moved = df_multiple.copy(deep=True)
    df_multiple_moved['joinIndex'] = df_multiple_moved.index - 1
    df_multiple_join = df_multiple.merge(df_multiple_moved, how='inner', on=['joinIndex', 'License_plate'],
                                         suffixes=['_from', '_to'])

    display(df_multiple)
    display(df_multiple_moved)

    display(df_multiple_join)

    crosstab = pd.crosstab(df_multiple_join['Direction_from'],
                           df_multiple_join['Direction_to'])
    crosstab = crosstab - 1  # remove fake data

    return df_multiple

In [72]:
dfm = computeData(df, params)

In [73]:
dfm

{'multiple': Direction_to    1     2     3     4     5    6     7    8     9     10    11   \
 Direction_from                                                                  
 1                27    56   982    32     1    5     1   10     5    77     6   
 2               231     8   120     2    49    0     2    1     7    14     2   
 3                 5    60   110  1408     4  182     9  273    26  1688     8   
 4               257  1286  1155    53   181    3    34   16    44    79    16   
 5                 5    20    13   287    71  941    11  403    12   552     5   
 6                69     2   207    13  1587   31   124   13    82    16    10   
 7                 9    20    35   278    17  341    37  710    21   471     2   
 8                15     6    30    23   124   13  1031   71    99    28    61   
 9                89    87   278  1880    61  416    32  619    56  2577    18   
 10               12    17   112    20   117    8    64   27   515    50   422   
 11 

In [78]:
[str(i + 1) for i in range(N * 2)]


['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18']

In [ ]:
fake_df = {'Capture_time': [], 'License_plate': [], 'Direction': []}
fake_end_time = pd.to_datetime('23:59')

xd = pd.DataFrame.from_dict(fake_df)

for i in range(1, N * 2 + 1):
    fake_df['Capture_time'].append(fake_start_time)
    fake_df['License_plate'].append(f'fake_{i}')
    fake_df['Direction'].append(i)

    for k in range(1, N * 2 + 1):
        fake_df['Capture_time'].append(fake_start_time)
        fake_df['License_plate'].append(f'fake_{i}_{k}')
        fake_df['Direction'].append(i)
        fake_df['Capture_time'].append(fake_end_time)
        fake_df['License_plate'].append(f'fake_{i}_{k}')
        fake_df['Direction'].append(k)

return pd.DataFrame.from_dict(fake_df)

In [32]:
fake_df = {'Capture_time': [], 'License_plate': [], 'Direction': []}
fake_end_time = pd.to_datetime('23:59')

xd = pd.DataFrame.from_dict(fake_df)

In [33]:
xd[]

,Capture_time,License_plate,Direction


In [52]:
endStopDf = pd.DataFrame.from_dict({'Capture_time': [], 'License_plate': [], 'Direction': []})

endStopDf['License_plate'] = df['License_plate'].unique()
endStopDf['Direction'] = -1
endStopDf['Capture_time'] = pd.to_datetime('23:59')
endStopDf

,Capture_time,License_plate,Direction
0,2021-11-14 23:59:00,2SH9059,-1
1,2021-11-14 23:59:00,1SZ2343,-1
2,2021-11-14 23:59:00,7A14682,-1
3,2021-11-14 23:59:00,unknown,-1
4,2021-11-14 23:59:00,4SM3348,-1
...,...,...,...
36414,2021-11-14 23:59:00,46F7919,-1
36415,2021-11-14 23:59:00,BAP12670,-1
36416,2021-11-14 23:59:00,AXT8338,-1
36417,2021-11-14 23:59:00,IUC8127,-1


In [ ]:
endStopDf = endStopDf[endStopDf['License_plate']]

In [50]:
pd.Series(dfm['License_plate'].unique()).str.contains('fake_')

0        False
1        False
2        False
3        False
4        False
         ...  
22087     True
22088     True
22089     True
22090     True
22091     True
Length: 22092, dtype: bool

In [56]:
endStopDf['License_plate'].str.contains('fake_')

0        False
1        False
2        False
3        False
4        False
         ...  
36414    False
36415    False
36416    False
36417    False
36418    False
Name: License_plate, Length: 36419, dtype: bool

In [20]:
df.groupby(by='Direction').count().drop(columns=['License_plate'])

,Capture_time
Direction,
1,2668
2,2868
3,8149
4,7390
5,4311
6,4173
7,3463
8,3680
9,8815
